In [6]:
import itertools
import re
import nltk
from nltk.corpus import stopwords
import networkx as nx
import matplotlib.pyplot as plt
import unicodedata

from nltk import bigrams, FreqDist
import string

from collections import Counter
import pandas as pd
import numpy as np
from matplotlib.patches import Patch
import glob
import os

from dateutil.parser import parse
import dateutil.parser as parser

In [7]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
df_TikTok= pd.read_excel('/content/Peru_tiktok_clean.xlsx')

# Co_hashtag TikTok

In [4]:
df_TikTok['videoId'] = df_TikTok['webVideoUrl'].apply(lambda url: url.split("/")[-1].split("?")[0])

In [5]:
df_TikTok.head()

,Unnamed: 0,id,text,createTime,createTimeISO,locationCreated,isAd,isMuted,authorMeta,musicMeta,...,videoMeta,diggCount,shareCount,playCount,commentCount,mentions,hashtags,effectStickers,searchHashtag,videoId
0,0,7178572463171800064,DO NOT TRAVEL PERU NOW I didn’t want to make t...,1671391653,2022-12-18T19:27:33.000Z,BO,False,False,"{'id': '6813482726567986181', 'name': 'will_th...",{'musicName': 'Chopin Nocturne No. 2 Piano Mon...,...,"{'height': 1024, 'width': 576, 'duration': 133...",1700000,28000,13200000,18300,[],"[{'id': '17669', 'name': 'peru', 'title': '', ...",[],"{'views': '14.3M', 'name': 'peruprotests'}",7178572463171800321
1,1,7178910881479199744,Replying to @Josh Ambrose This is how I escap...,1671470450,2022-12-19T17:20:50.000Z,BO,False,False,"{'id': '6813482726567986181', 'name': 'will_th...",{'musicName': 'Chopin Nocturne No. 2 Piano Mon...,...,"{'height': 1024, 'width': 576, 'duration': 44,...",84400,511,871700,1044,['@Josh'],"[{'id': '17669', 'name': 'peru', 'title': '', ...",[],"{'views': '14.3M', 'name': 'peruprotests'}",7178910881479200002
2,2,7189529619882609664,URGENTE: L0s 4 detenid0s en Humay fueron liber...,1673942823,2023-01-17T08:07:03.000Z,PE,False,False,"{'id': '7177521712358343685', 'name': 'agendap...","{'musicName': 'sonido original', 'musicAuthor'...",...,"{'height': 868, 'width': 480, 'duration': 45, ...",5701,245,45900,419,[],"[{'id': '33490496', 'name': 'humay', 'title': ...",[],"{'views': '14.3M', 'name': 'peruprotests'}",7189529619882609925
3,3,7183865473107233792,Crossing into peru with the protests going on…...,1672624030,2023-01-02T01:47:10.000Z,CO,False,False,"{'id': '7079127045952095237', 'name': 'fly5int...","{'musicName': 'love nwantinti (ah ah ah)', 'mu...",...,"{'height': 1024, 'width': 576, 'duration': 62,...",4394,3,33700,38,[],"[{'id': '205020', 'name': 'aviation', 'title':...",[],"{'views': '14.3M', 'name': 'peruprotests'}",7183865473107234054
4,4,7191282491183975424,Me da asco la desinformación que hay en todos ...,1674350938,2023-01-22T01:28:58.000Z,PE,False,False,"{'id': '6937847160369349637', 'name': 'camyder...","{'musicName': 'Love You So', 'musicAuthor': 'T...",...,"{'height': 576, 'width': 576, 'duration': 11, ...",3875,75,30000,1469,[],"[{'id': '1637342470396934', 'name': 'fypシ', 't...",[],"{'views': '14.3M', 'name': 'peruprotests'}",7191282491183975685


In [8]:
df_TikTok['text'] = df_TikTok['text'].astype(str)

In [9]:
# Extract hashtags from the text column using regular expressions
df_TikTok['thehashtags'] = df_TikTok['text'].apply(lambda x: re.findall(r'#(\w+)', x))

In [10]:
# Create a DataFrame to store co-occurring hashtags, video IDs, and date
co_occurrence_df = pd.DataFrame(columns=['Co-occurring Hashtags', 'Video IDs', 'Date'])

for index, row in df_TikTok.iterrows():
    video_id = row['videoId']
    hashtags = row['thehashtags']
    date = row['createTimeISO']

    # Find co-occurring hashtags in each row
    row_bigrams = list(bigrams(hashtags))
    row_bigram_counts = Counter(row_bigrams)

    for bigram, count in row_bigram_counts.items():
        sorted_bigram = tuple(sorted(bigram))  # Sort the hashtags within the bigram
        co_occurrence_df = co_occurrence_df.append({'Co-occurring Hashtags': sorted_bigram,
                                                    'Video IDs': video_id,
                                                    'Date': date}, ignore_index=True)


Streaming output truncated to the last 5000 lines.
<ipython-input-10-6679e0e4513d>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  co_occurrence_df = co_occurrence_df.append({'Co-occurring Hashtags': sorted_bigram,
<ipython-input-10-6679e0e4513d>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  co_occurrence_df = co_occurrence_df.append({'Co-occurring Hashtags': sorted_bigram,
<ipython-input-10-6679e0e4513d>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  co_occurrence_df = co_occurrence_df.append({'Co-occurring Hashtags': sorted_bigram,
<ipython-input-10-6679e0e4513d>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  co_occurrence

In [11]:
co_occurrence_df

,Co-occurring Hashtags,Video IDs,Date
0,"(peru, perutravel)",7178572463171800321,2022-12-18T19:27:33.000Z
1,"(backpack, perutravel)",7178572463171800321,2022-12-18T19:27:33.000Z
2,"(backpack, backpacking)",7178572463171800321,2022-12-18T19:27:33.000Z
3,"(backpacking, travel)",7178572463171800321,2022-12-18T19:27:33.000Z
4,"(travel, travelstorys)",7178572463171800321,2022-12-18T19:27:33.000Z
...,...,...,...
58335,"(chalaarequipa, protestaperú)",7179198317522472197,2022-12-20T11:56:12.000Z
58336,"(protestaperú, viral)",7179198317522472197,2022-12-20T11:56:12.000Z
58337,"(ayacucho, protestaperú)",7177761257603206405,2022-12-16T14:59:36.000Z
58338,"(ayacucho, viral)",7177761257603206405,2022-12-16T14:59:36.000Z


In [12]:
co_occurrence_df.to_csv('TikTok_Hashtag_Peru.csv', index=False)

# Co-words TikTok

In [13]:
#df['videoId'] = df['webVideoUrl'].apply(lambda url: url.split("/")[-1].split("?")[0])
#df_TikTok['text'] = df_TikTok['text'].astype(str)
#df_TikTok['translated_text'] = df_TikTok['translated_text'].astype(str)

In [14]:
def preprocess_text(text):

    # Remove mentions
    user_pattern = r'@[A-Za-z0-9_]+'
    # Remove twitter hashtags
    hashtag_pattern = r'#\w+\b'
    # Remove usernames
    text = re.sub(user_pattern, '', text)
    text = re.sub(hashtag_pattern, " ", text)

    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)

    # Convert to lowercase
    text = text.lower()

    # Remove non-alphabetic tokens
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Remove stopwords
    stop_words = set(stopwords.words(['english', 'spanish', 'portuguese']))
    tokens = nltk.word_tokenize(text)
    filtered_tokens = [word for word in tokens if word not in stop_words]
    text = ' '.join(filtered_tokens)

    # # Remove hashtags and mentions
    # text = re.sub(r'#[\w_]+', '', text)
    # text = re.sub(r'@[\w_]+', '', text)
    # Remove hashtags and mentions
    # Remove emoji
    text = remove_emoji(text)


    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)

    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()

    return text

def remove_emoji(text):
    # Remove emoji using Unicode patterns
    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"  # Emoticons
        "\U0001F300-\U0001F5FF"  # Symbols & Pictographs
        "\U0001F680-\U0001F6FF"  # Transport & Map Symbols
        "\U0001F1E0-\U0001F1FF"  # Flags (iOS)
        "\U00002702-\U000027B0"
        "\U000024C2-\U0001F251"
        "]+",
        flags=re.UNICODE,
    )
    return emoji_pattern.sub(r'', text)

In [15]:
# Apply preprocessing
preprocessed_texts = []
for index, row in df_TikTok.iterrows():
    text = row['text']
    preprocessed_text = preprocess_text(text)
    preprocessed_texts.append(preprocessed_text)

df_TikTok['preprocessed_text'] = preprocessed_texts

In [16]:
# Tokenization and Bigram extraction for the 'preprocessed_text' column
all_bigrams = []
for _, row in df_TikTok.iterrows():
    video_id = row['videoId']
    date = row['createTimeISO']
    sentence = row['preprocessed_text']
    tokens = nltk.word_tokenize(sentence)
    bigrams = list(nltk.bigrams(tokens))
    for bigram in bigrams:
        all_bigrams.append((bigram, video_id, date))

output_data = pd.DataFrame(all_bigrams, columns=['Word Pair','video_id', 'date'])

# Save the DataFrame as a CSV file
#output_data.to_csv('bigram_pairs.csv', index=False)

In [17]:
output_data

,Word Pair,video_id,date
0,"(travel, peru)",7178572463171800321,2022-12-18T19:27:33.000Z
1,"(peru, didnt)",7178572463171800321,2022-12-18T19:27:33.000Z
2,"(didnt, want)",7178572463171800321,2022-12-18T19:27:33.000Z
3,"(want, make)",7178572463171800321,2022-12-18T19:27:33.000Z
4,"(make, video)",7178572463171800321,2022-12-18T19:27:33.000Z
...,...,...,...
17119,"(vimos, caer)",7190873204775259397,2023-01-20T23:00:42.000Z
17120,"(caer, felizmente)",7190873204775259397,2023-01-20T23:00:42.000Z
17121,"(felizmente, delante)",7190873204775259397,2023-01-20T23:00:42.000Z
17122,"(delante, auto)",7190873204775259397,2023-01-20T23:00:42.000Z


In [18]:
output_data.to_csv('TikTok_Words_Peru.csv', index=False)